In [1]:
import numpy as np
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import multiprocessing as mp
from oucorr import calcCorr

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

SEED = 350107321 # from random.org
np.random.seed(SEED)

print(plt.style.available)
plt.style.use('seaborn-white')

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [2]:
roi_time_series = pd.read_csv("sub-021_ses-bhb_task-rest_run-1.csv",header=None, index_col=False)

    # Convert into numpy array
roi_time_series = np.array(roi_time_series)
roi_time_series.shape

(720, 498)

In [3]:
# 720 = 24x30 so splitting it up into chuncks of 30 is a good idea
WINDOW_LENGTH = 30
REGIONS = 498
num = int((roi_time_series.shape[0] - 1 - \
               (roi_time_series.shape[0] - 1) % WINDOW_LENGTH) /
              WINDOW_LENGTH)
indicies = np.linspace(WINDOW_LENGTH, num*WINDOW_LENGTH, num).astype(int)

windows = np.split(roi_time_series, indicies, axis=0)

In [4]:
windows[0].shape

(30, 498)

In [5]:
C,dC,B1 = calcCorr(windows,WINDOW_LENGTH,REGIONS)

/Users/hstrey/Documents/programming/fMRI-analysis/oucorr.py:29: RuntimeWarning: invalid value encountered in true_divide
  return ((3.0 * c / a) - ((b ** 2.0) / (a ** 2.0))) / 3.0
/Users/hstrey/Documents/programming/fMRI-analysis/oucorr.py:34: RuntimeWarning: invalid value encountered in true_divide
  return (((2.0 * (b ** 3.0)) / (a ** 3.0)) - ((9.0 * b * c) / (a **2.0)) + (27.0 * d / a)) /27.0
/Users/hstrey/Documents/programming/fMRI-analysis/oucorr.py:20: RuntimeWarning: invalid value encountered in true_divide
  P = (b / (3.0 * a)) * -1                # Helper Temporary Variable
/Users/hstrey/Documents/programming/fMRI-analysis/oucorr.py:22: RuntimeWarning: invalid value encountered in true_divide
  x1 = 2 * j * np.cos(k / 3.0) - (b / (3.0 * a))


In [6]:
C[0]

array([[         nan,  -1.58542928,   1.87447183, ...,   4.05409717,
          2.31976218,   0.88293619],
       [ -1.58542928,          nan, -16.89688348, ...,  -2.4310583 ,
         -4.54571756,  -9.76774106],
       [  1.87447183, -16.89688348,          nan, ...,   6.06607665,
         20.37429626,  12.91161931],
       ...,
       [  4.05409717,  -2.4310583 ,   6.06607665, ...,          nan,
          6.5939766 ,   1.47102682],
       [  2.31976218,  -4.54571756,  20.37429626, ...,   6.5939766 ,
                 nan,   4.20298531],
       [  0.88293619,  -9.76774106,  12.91161931, ...,   1.47102682,
          4.20298531,          nan]])

In [7]:
dC[0]

array([[        nan,  3.00790293,  2.42693685, ...,  5.24666676,
         2.60125193,  1.59845866],
       [ 3.00790293,         nan, 21.89639434, ...,  4.96359135,
         6.52193515, 10.78727376],
       [ 2.42693685, 21.89639434,         nan, ...,  5.92966089,
        15.65464668, 13.58369068],
       ...,
       [ 5.24666676,  4.96359135,  5.92966089, ...,         nan,
         5.47624359,  2.39665534],
       [ 2.60125193,  6.52193515, 15.65464668, ...,  5.47624359,
                nan,  4.66428864],
       [ 1.59845866, 10.78727376, 13.58369068, ...,  2.39665534,
         4.66428864,         nan]])

In [8]:
B1[0]

array([[0.97152868, 0.95703125, 0.96033793, ..., 0.97357296, 0.96225001,
        0.94030259],
       [0.95703125, 0.94790802, 0.92336116, ..., 0.98210299, 0.91820517,
        0.91398892],
       [0.96033793, 0.92336116, 0.95190829, ..., 0.96274148, 0.94895727,
        0.94259002],
       ...,
       [0.97357296, 0.98210299, 0.96274148, ..., 0.97242189, 0.96519965,
        0.94717637],
       [0.96225001, 0.91820517, 0.94895727, ..., 0.96519965, 0.94272278,
        0.93836688],
       [0.94030259, 0.91398892, 0.94259002, ..., 0.94717637, 0.93836688,
        0.9367229 ]])